# **Librerias**

In [1]:
# Importamos las Librerias 

import matplotlib
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Versiones 

print(f'Numpy Version: {np.__version__}')
print(f'CVXPY Version: {cp.__version__}')
print(f'Matplotlib Version: {matplotlib.__version__}')

Numpy Version: 2.3.1
CVXPY Version: 1.7.1
Matplotlib Version: 3.10.3


# **Lasso**

Conocemos que podemos calcular el parametro de **maximo a posteriori** $w$, como un problema de **optimizacion convexa** utilizando la funcion de **Negative Log-Verosimilitud**:

$$NLL(w \mid y, X, \sigma^2, b)  = D \log \left(2b\right) + \frac{1}{b} ||w||_1 + \frac{n}{2} \log \left(2\pi\sigma^2\right) + \frac{1}{2 \sigma^2} \cdot (y - Xw)^T (y - Xw)$$

$$\text{Loss} = \frac{1}{b} ||w||_1 + \frac{1}{2 \sigma^2} \cdot (y - Xw)^T (y - Xw)$$
$$\text{Loss} = \frac{\sigma^2}{b} ||w||_1 + \frac{1}{2} (y - Xw)^T (y - Xw)$$

$$\begin{align*}
\text{minimizar}_w \quad & \frac{1}{2} ||y - Xw||_2^2 + \lambda ||w||_1
\end{align*}$$

En donde 

$$\lambda = \frac{\sigma^2}{b}$$

* $\sigma^2$ grande, tenemos mas confianza en el **prior** ya que los datos son ruidosos 

* $b$ grande, tenemos mas confianza en el **likehood** ya que el prior es muy laxo 

* $b$ pequeño, tenemos mas confianza en el **prior** ya que el prior es muy restrictivo

El mismo problema se puede expresar como una **restricción sobre el conjunto de soluciones factibles**:

$$\begin{align*}
\text{minimizar} \quad & \frac{1}{2} ||y - Xw||_2^2  \\ \text{sujeto a} \quad & ||w||_1 - t \leq 0
\end{align*}$$

En donde 

* $t$ grande, tenemos mas confianza en el **likehood** ya que el prior es muy laxo 

* $t$ pequeño, tenemos mas confianza en el **prior** ya que el prior es muy restrictivo

Otra forma de interpretarlo 

* $t$ grande, restricción débil, solución cercana a mínimos cuadrados (OLS).

* $t$ pequeño, restricción fuerte, los pesos se contraen hacia 0, promoviendo sparsity.

In [2]:
# Semilla

np.random.seed(25)

# Definimos el Dataset

X, y = make_regression(n_samples = 100, n_features = 2, noise = 10.0, random_state = 25)

# Definimos la grilla de la Funcion de Perdidas (Minimos Cuadrados)

w1_vals = np.linspace(-200, 400, 300)
w2_vals = np.linspace(-200, 400, 300)

W1, W2 = np.meshgrid(w1_vals, w2_vals)

Loss = np.zeros_like(W1)

for w1_i in range(0, W1.shape[0]):
    for w2_i in range(0, W1.shape[1]):

        w_i = np.array([W1[w1_i, w2_i], W2[w1_i, w2_i]])
        b_i = np.mean(y - X @ w_i)

        Loss[w1_i, w2_i] = np.sum((y - (X @ w_i + b_i))**2)

# Calculamos el Punto Optimo de la Funcion de Perdidas sin Regularizacion (Minimos Cuadrados)

model_least_square = LinearRegression()

model_least_square.fit(X, y)

w_opt_least_square = model_least_square.coef_
b_opt_least_square = model_least_square.intercept_

loss_least_square = np.sum((y - (X @ w_opt_least_square + b_opt_least_square))**2)

# Funcion Interactiva que demuestra la Regularizacion 

def viz_lasso(t, elev = 30, azim = 45): 
    
    X_with_bias = np.hstack([np.ones((X.shape[0], 1)), X])

    weights = cp.Variable(3)
    constraints = [cp.norm1(weights[1:]) <= t]
    objective = cp.Minimize(0.5 * cp.sum_squares(y - X_with_bias @ weights))

    problem = cp.Problem(objective, constraints)
    problem.solve()

    w_opt_lasso = weights.value[1:]
    b_opt_lasso = weights.value[0]

    loss_lasso = np.sum((y - (X @ w_opt_lasso + b_opt_lasso))**2)

    norm_w1 = np.array([0, t, 0, -t, 0])
    norm_w2 = np.array([t, 0, -t, 0, t])

    norm = np.zeros_like(norm_w1)
    for idx in range(len(norm_w1)):
        w_c = np.array([norm_w1[idx], norm_w2[idx]])
        b_c = np.mean(y - X @ w_c)
        norm[idx] = np.sum((y - (X @ w_c + b_c))**2)

    fig = plt.figure(figsize = (10 , 8))
    ax = fig.add_subplot(111, projection = '3d')
    
    ax.plot_surface(W1, W2, Loss, alpha = 0.5, cmap = 'inferno')
    ax.plot(norm_w1, norm_w2, norm, color = 'blue', linewidth = 3, label = f'Norma L1 t = {t:.2f}')
    ax.plot_trisurf(norm_w1, norm_w2, np.zeros_like(norm_w1), color = "blue", alpha = 0.3)
    
    ax.scatter(w_opt_lasso[0], w_opt_lasso[1], loss_lasso, color = 'black', s = 20, label = 'Óptimo Lasso')
    ax.scatter(w_opt_least_square[0], w_opt_least_square[1], loss_least_square, color = 'green', s = 20, label = 'Óptimo Minimos Cuadrados')
    
    ax.view_init(elev = elev, azim = azim)    
    ax.set_xlabel(r'$w_1$')
    ax.set_ylabel(r'$w_2$')
    ax.set_title(f'Loss Minimos Cuadrados {loss_least_square:.2f}, Loss Lasso {loss_lasso:.2f}\nMinimos Cuadrados $\\rightarrow$ $w_1$ = {w_opt_least_square[0]:.2f} $w_2$ = {w_opt_least_square[1]:.2f}\nLasso $\\rightarrow$ $w_1$ = {w_opt_lasso[0]:.2f} $w_2$ = {w_opt_lasso[1]:.2f}')
    ax.legend()
    
    plt.show()

t_max = np.linalg.norm(w_opt_least_square, ord = 1)

t_slider = widgets.FloatSlider(value = t_max, min = 0.1*t_max, max = t_max, step = 0.05*t_max, description = 't')
elev_slider = widgets.IntSlider(value = 30, min = 0, max = 90, step = 1, description = 'Elevacion')
azim_slider = widgets.IntSlider(value = 45, min = 0, max = 360, step = 1, description = 'Azimut')

widgets.interact(viz_lasso, t = t_slider, elev = elev_slider, azim = azim_slider)
plt.show()

interactive(children=(FloatSlider(value=102.57229603433198, description='t', max=102.57229603433198, min=10.25…

# **Ridge**

Conocemos que podemos calcular el parametro de **maximo a posteriori** $w$, como un problema de **optimizacion convexa** utilizando la funcion de **Negative Log-Verosimilitud**:

$$NLL(w \mid y, X, \sigma^2, \tau^2) = \frac{D}{2} \log 2 \pi \tau^2 + \frac{1}{2\tau^2} w^Tw  + \frac{n}{2} \log 2\pi\sigma^2 + \frac{1}{2 \sigma^2} \cdot (y - Xw)^T (y - Xw)$$

$$\text{Loss} = \frac{1}{2\tau^2} w^Tw + \frac{1}{2 \sigma^2} \cdot (y - Xw)^T (y - Xw)$$
$$\text{Loss} = \frac{\sigma^2}{\tau^2} w^Tw + (y - Xw)^T (y - Xw)$$

$$\begin{align*}
\text{minimizar}_w \quad & ||y - Xw||_2^2 + \lambda ||w||_2^2 
\end{align*}$$

En donde 

$$\lambda = \frac{\sigma^2}{\tau^2}$$

* $\sigma^2$ grande, tenemos mas confianza en el **prior** ya que los datos son ruidosos 

* $\tau^2$ grande, tenemos mas confianza en el **likehood** ya que el prior es muy laxo 

* $\tau^2$ pequeño, tenemos mas confianza en el **prior** ya que el prior es muy restrictivo

El mismo problema se puede expresar como una **restricción sobre el conjunto de soluciones factibles**:

$$\begin{align*}
\text{minimizar} \quad & ||y - Xw||_2^2  \\ \text{sujeto a} \quad & ||w||_2 - t \leq 0
\end{align*}$$

En donde 

* $t$ grande, tenemos mas confianza en el **likehood** ya que el prior es muy laxo 

* $t$ pequeño, tenemos mas confianza en el **prior** ya que el prior es muy restrictivo

Otra forma de interpretarlo 

* $t$ grande, restricción débil, solución cercana a mínimos cuadrados (OLS).

* $t$ pequeño, restricción fuerte, los pesos se contraen hacia valores pequeños, pero muy raramente hacia 0.

In [3]:
# Semilla

np.random.seed(25)

# Definimos el Dataset

X, y = make_regression(n_samples = 100, n_features = 2, noise = 10.0, random_state = 25)

# Definimos la grilla de la Funcion de Perdidas (Minimos Cuadrados)

w1_vals = np.linspace(-200, 400, 300)
w2_vals = np.linspace(-200, 400, 300)

W1, W2 = np.meshgrid(w1_vals, w2_vals)

Loss = np.zeros_like(W1)

for w1_i in range(0, W1.shape[0]):
    for w2_i in range(0, W1.shape[1]):

        w_i = np.array([W1[w1_i, w2_i], W2[w1_i, w2_i]])
        b_i = np.mean(y - X @ w_i)

        Loss[w1_i, w2_i] = np.sum((y - (X @ w_i + b_i))**2)

# Calculamos el Punto Optimo de la Funcion de Perdidas sin Regularizacion (Minimos Cuadrados)

model_least_square = LinearRegression()

model_least_square.fit(X, y)

w_opt_least_square = model_least_square.coef_
b_opt_least_square = model_least_square.intercept_

loss_least_square = np.sum((y - (X @ w_opt_least_square + b_opt_least_square))**2)

# Funcion Interactiva que demuestra la Regularizacion 

def viz_ridge(t, elev = 30, azim = 45): 
    
    X_with_bias = np.hstack([np.ones((X.shape[0], 1)), X])

    weights = cp.Variable(3)
    constraints = [cp.norm2(weights[1:]) <= t]
    objective = cp.Minimize(cp.sum_squares(y - X_with_bias @ weights))

    problem = cp.Problem(objective, constraints)
    problem.solve()

    w_opt_ridge = weights.value[1:]
    b_opt_ridge = weights.value[0]

    loss_ridge = np.sum((y - (X @ w_opt_ridge + b_opt_ridge))**2)

    theta = np.linspace(0, 2*np.pi, 200)
    norm_w1 = t * np.cos(theta)
    norm_w2 = t * np.sin(theta)

    norm = np.zeros_like(norm_w1)
    for idx in range(len(norm_w1)):
        w_c = np.array([norm_w1[idx], norm_w2[idx]])
        b_c = np.mean(y - X @ w_c)
        norm[idx] = np.sum((y - (X @ w_c + b_c))**2)

    fig = plt.figure(figsize = (10 , 8))
    ax = fig.add_subplot(111, projection = '3d')
    
    ax.plot_surface(W1, W2, Loss, alpha = 0.5, cmap = 'inferno')
    ax.plot(norm_w1, norm_w2, norm, color = 'blue', linewidth = 3, label = f'Norma L2 t = {t:.2f}')
    ax.plot_trisurf(norm_w1, norm_w2, np.zeros_like(norm_w1), color = "blue", alpha = 0.3)
    
    ax.scatter(w_opt_ridge[0], w_opt_ridge[1], loss_ridge, color = 'black', s = 20, label = 'Óptimo Ridge')
    ax.scatter(w_opt_least_square[0], w_opt_least_square[1], loss_least_square, color = 'green', s = 20, label = 'Óptimo Minimos Cuadrados')
    
    ax.view_init(elev = elev, azim = azim)    
    ax.set_xlabel(r'$w_1$')
    ax.set_ylabel(r'$w_2$')
    ax.set_title(f'Loss Minimos Cuadrados {loss_least_square:.2f}, Loss Ridge {loss_ridge:.2f}\nMinimos Cuadrados $\\rightarrow$ $w_1$ = {w_opt_least_square[0]:.2f} $w_2$ = {w_opt_least_square[1]:.2f}\nRidge $\\rightarrow$ $w_1$ = {w_opt_ridge[0]:.2f} $w_2$ = {w_opt_ridge[1]:.2f}')
    ax.legend()
    
    plt.show()

t_max = np.linalg.norm(w_opt_least_square, ord = 2)

t_slider = widgets.FloatSlider(value = t_max, min = 0.1*t_max, max = t_max, step = 0.05*t_max, description = 't')
elev_slider = widgets.IntSlider(value = 30, min = 0, max = 90, step = 1, description = 'Elevacion')
azim_slider = widgets.IntSlider(value = 45, min = 0, max = 360, step = 1, description = 'Azimut')

widgets.interact(viz_ridge, t = t_slider, elev = elev_slider, azim = azim_slider)
plt.show()

interactive(children=(FloatSlider(value=96.63766353897923, description='t', max=96.63766353897923, min=9.66376…